In [3]:
search_for = 126
start_from = 0
threads = 10
thread = 8
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0020275115966796875
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 375
all cases: 956096141
type: [1, 1, 1, 1, 126] 126
cases of this type: 252047376
100000 22.045201945453766
200000 35.21242178381278
300000 42.326621897153835
400000 39.86965455834654
500000 53.85558759877837
600000 51.52788300010951
700000 50.318202789999674
800000 49.37113562361618
900000 54.65635231501312
1000000 41.10420629173484
1100000 34.23360842409558
1200000 56.36383770668484
1300000 57.40150151426312
1400000 55.99649236239599
1500000 53.82377671242363
1600000 57.90959534735635
1700000 57.99242539089396
1800000 55.524019500487356
1900000 60.9344453276987
2000000 38.5941450232019
2100000 25.157560758201964
2200000 41.31076718370054
2300000 49.96086349302161
2400000 46.941883196680706
2500000 51.846174623770096
2600000 47.61649912435491
2700000 47.66854549418942
2800000 51.666193877137296
2900000 54.97941426280064
3000000 43.83024565

30100000 74.37180795420836
30200000 73.40802766456154
30300000 31.39595593776458
30400000 79.04557929265295
30500000 74.80018310762486
30600000 73.78400356121249
30700000 70.52496401458342
30800000 69.98036153342291
30900000 64.93599931690626
31000000 53.05627483673261
31100000 41.91312740154688
31200000 65.39073453882925
31300000 68.08348652735825
31400000 71.00849602748286
31500000 70.21153256117422
31600000 74.55632155602514
31700000 75.44632447365062
31800000 41.295861481951995
31900000 65.44390002427251
32000000 73.67305742568738
32100000 77.310197411136
32200000 77.12868347807078
32300000 37.98983473634138
32400000 72.4828090049202
32500000 76.30352277458995
32600000 72.22309827927391
32700000 71.91890160658137
32800000 67.86996003817923
32900000 71.25659117396026
33000000 56.34106043589167
33100000 40.27311451166503
33200000 69.29712077674192
33300000 66.91832186646309
33400000 69.70781488296352
33500000 74.45216935928705
33600000 75.68881207431943
33700000 72.83301030511308
338

60600000 51.4512929609061
60700000 65.52600161704551
60800000 66.42815519454929
60900000 65.31402971265646
61000000 55.960653737955255
61100000 35.64178367341283
61200000 63.71317184748887
61300000 64.1269387257331
61400000 64.40749406210955
61500000 66.3142565427362
61600000 33.42071221238199
61700000 67.94890851772625
61800000 72.43750671184468
61900000 69.72094578817513
62000000 77.80075005689683
62100000 74.97226533704021
62200000 73.94908893561535
62300000 69.74398026256647
62400000 70.35133293749242
62500000 54.60421392925936
62600000 43.289652893262485
62700000 65.36585301002894
62800000 64.05281204676956
62900000 61.96930931184086
63000000 52.592268413818125
63100000 34.45516013729041
63200000 62.83818870035016
63300000 62.940169919788495
63400000 67.28566680531682
63500000 60.3194604788346
63600000 40.34761604833446
63700000 68.07505663338871
63800000 69.95323404962286
63900000 68.5256778255602
64000000 71.06182014575542
64100000 82.5943723988102
64200000 68.73369577346699
643

91000000 52.467180570688214
91100000 32.00607668470964
91200000 57.78501252795932
91300000 48.553251217399996
91400000 44.93292956688811
91500000 61.31938098750839
91600000 62.492359178065946
91700000 62.90202030939826
91800000 63.9021130572285
91900000 66.87288434751952
92000000 64.82441753708714
92100000 67.93901349375199
92200000 70.19270172291283
92300000 68.2258383825926
92400000 64.39203095382038
92500000 64.09402577181284
92600000 60.18135198330796
92700000 60.450134143063565
92800000 27.096517781031164
92900000 58.26897563957504
93000000 54.92294766990805
93100000 31.12518717694403
93200000 57.73074547642913
93300000 41.048544308477815
93400000 49.26650920430605
93500000 59.8443574395078
93600000 59.80074700057518
93700000 66.7278991709968
93800000 65.7054849563818
93900000 67.34742034474431
94000000 66.90241989536233
94100000 65.08619607974562
94200000 66.4267138402023
94300000 66.27838549391551
94400000 61.36436250081447
94500000 66.22751193832796
94600000 59.96073560849107
9

120400000 74.6987606129876
120500000 74.35056092630346
120600000 74.90616746008791
120700000 70.9120104274151
120800000 73.42107324958302
120900000 75.37857520787016
121000000 37.46929581308998
121100000 10.987053565515696
121200000 63.442589443863255
121300000 70.06193149581364
121400000 73.00689157902333
121500000 74.61555091411152
121600000 75.00371694680513
121700000 74.11526982322921
121800000 76.29363529625641
121900000 78.10877403971273
122000000 79.46512754573935
122100000 24.668784842348984
122200000 6.6408178707771555
122300000 6.709174791741153
122400000 6.52104007194093
122500000 6.559571266797859
122600000 6.4203972939602645
122700000 6.787264142279455
122800000 7.0861555520350645
122900000 6.968865702522451
123000000 6.590929774968403
123100000 7.083789783816611
123200000 6.442902357190792
123300000 6.545838559058065
123400000 6.346831697262151
123500000 7.057979574056144
123600000 6.441612126554561
123700000 6.468291809962901
123800000 7.053355090451967
123900000 6.32250

...

884700000 2.4251425277052348
884800000 3.4826844607293563
884900000 2.159875577312813
885000000 3.0995012049510247
885100000 2.2682624015449795
885200000 2.997329234217328
885300000 3.086694934875844
885400000 2.305866586497275
885500000 3.1825857897297882
885600000 2.067080260591424
885700000 3.448920844074455
885800000 1.6945071357834418
885900000 3.2867249375091254
886000000 1.785608621757263
886100000 2.9549188183355333
886200000 2.5631203310093142
886300000 2.509685911711094
886400000 3.991938552722142
886500000 2.4548197606520863
886600000 1.6927273323120886
886700000 0.5296815298164328
886800000 0.5216908869157652
886900000 0.5200151843257116
887000000 0.5275488546642065
887100000 0.524017054811224
887200000 0.5173868263965792
887300000 0.5145185993172208
887400000 0.5323940390705963
887500000 0.5241624812622534
887600000 0.5185448533515652
887700000 2.737892809575963
887800000 2.441252673357924
887900000 4.170983802541226
888000000 2.0911086816602826
888100000 3.276669120456180

best so far: 0
type: [1, 3, 7, 1, 6] 126
cases of this type: 190512
909300000 0.9726931799671259
909400000 0.4465559542242759
best so far: 0
type: [1, 3, 7, 2, 3] 126
cases of this type: 95256
909500000 0.34017975977697373
best so far: 0
type: [1, 3, 7, 3, 2] 126
cases of this type: 63504
909600000 0.3242532051926149
best so far: 0
type: [1, 3, 7, 6, 1] 126
cases of this type: 31752
best so far: 0
type: [1, 3, 14, 1, 3] 126
cases of this type: 47628
909700000 0.31656188313758243
best so far: 0
type: [1, 3, 14, 3, 1] 126
cases of this type: 15876
best so far: 0
type: [1, 3, 21, 1, 2] 126
cases of this type: 21168
best so far: 0
type: [1, 3, 21, 2, 1] 126
cases of this type: 10584
best so far: 0
type: [1, 3, 42, 1, 1] 126
cases of this type: 5292
best so far: 0
type: [1, 6, 1, 1, 21] 126
cases of this type: 1166886
909800000 0.6175347941690207
909900000 0.8815879074505211
910000000 0.8776139759815302
910100000 0.8866067056276639
910200000 0.8751160073156595
910300000 0.3691203696100493
9

921100000 1.4236546456678343
921200000 0.8920290323071004
921300000 1.3894020717671718
921400000 0.9468933223048919
921500000 1.2256700267488816
921600000 1.3842538324311515
921700000 0.9177192919253236
921800000 1.4188278023148455
921900000 1.0725651741653985
922000000 1.3752974133636686
922100000 0.40908854189814325
922200000 0.2593697439343505
922300000 0.25116127578734293
922400000 0.24619443648542164
922500000 0.2502728870299246
922600000 0.24532580992188258
922700000 0.24392769837454553
922800000 0.24189609651632174
922900000 0.24150836349174382
923000000 0.24062844332145455
923100000 0.23955902159230907
923200000 0.2381197619015621
923300000 0.7645060775676648
923400000 1.1141873914835274
923500000 1.429280576322909
923600000 1.3535391846599678
923700000 0.8611847933007883
923800000 1.2766465115169876
923900000 0.8892086594547807
924000000 1.1320786191045191
924100000 1.2988693897877792
924200000 0.8209030649743835
924300000 1.2663026163362583
924400000 0.9242318326407465
924500

944800000 0.1911129340897858
944900000 0.13397867337544558
best so far: 0
type: [2, 3, 1, 7, 3] 126
cases of this type: 83349
945000000 0.12669761401347188
best so far: 0
type: [2, 3, 1, 21, 1] 126
cases of this type: 27783
best so far: 0
type: [2, 3, 3, 1, 7] 126
cases of this type: 64827
945100000 0.10500721752353707
best so far: 0
type: [2, 3, 3, 7, 1] 126
cases of this type: 9261
best so far: 0
type: [2, 3, 7, 1, 3] 126
cases of this type: 11907
best so far: 0
type: [2, 3, 7, 3, 1] 126
cases of this type: 3969
best so far: 0
type: [2, 3, 21, 1, 1] 126
cases of this type: 1323
best so far: 0
type: [2, 7, 1, 1, 9] 126
cases of this type: 45927
best so far: 0
type: [2, 7, 1, 3, 3] 126
cases of this type: 15309
945200000 0.10359512333291969
best so far: 0
type: [2, 7, 1, 9, 1] 126
cases of this type: 5103
best so far: 0
type: [2, 7, 3, 1, 3] 126
cases of this type: 5103
best so far: 0
type: [2, 7, 3, 3, 1] 126
cases of this type: 1701
best so far: 0
type: [2, 7, 9, 1, 1] 126
cases of t

955400000 0.009234251204127073
955500000 0.0070956806901945006
best so far: 0
type: [6, 1, 1, 3, 7] 126
cases of this type: 64827
best so far: 0
type: [6, 1, 1, 7, 3] 126
cases of this type: 27783
955600000 0.005592325860828824
best so far: 0
type: [6, 1, 1, 21, 1] 126
cases of this type: 9261
best so far: 0
type: [6, 1, 3, 1, 7] 126
cases of this type: 21609
best so far: 0
type: [6, 1, 3, 7, 1] 126
cases of this type: 3087
best so far: 0
type: [6, 1, 7, 1, 3] 126
cases of this type: 3969
best so far: 0
type: [6, 1, 7, 3, 1] 126
cases of this type: 1323
best so far: 0
type: [6, 1, 21, 1, 1] 126
cases of this type: 441
best so far: 0
type: [6, 3, 1, 1, 7] 126
cases of this type: 7203
best so far: 0
type: [6, 3, 1, 7, 1] 126
cases of this type: 1029
best so far: 0
type: [6, 3, 7, 1, 1] 126
cases of this type: 147
best so far: 0
type: [6, 7, 1, 1, 3] 126
cases of this type: 567
best so far: 0
type: [6, 7, 1, 3, 1] 126
cases of this type: 189
best so far: 0
type: [6, 7, 3, 1, 1] 126
cases 